# 1. Load data

In [2]:
import pandas as pd

In [155]:
cleanData = "simple.csv"
data = pd.read_csv(cleanData, sep=',')

In [157]:
data = data.ix[:, ['parcelid', 'latitude', 'longitude']]

In [158]:
data.head()

,parcelid,latitude,longitude
0,10754147,34144442,-118654084
1,10759547,34140430,-118625364
2,10843547,33989359,-118394633
3,10859147,34148863,-118437206
4,10879947,34194168,-118385816


In [13]:
data.size

3000

# 2. Pack rows into items

In [51]:
for attribute in data:
    print(attribute)

parcelid
latitude
longitude


In [52]:
for index, item in data.iterrows():
    if (index % 25 == 24):
        print(index, item['parcelid'], item['latitude'], item['longitude'])

24 11417147 33931500 -118352104
49 11818747 34072919 -118164004
74 12168547 34192420 -118125270
99 12560547 33770013 -118192643
124 13114547 34189273 -118545075
149 13119547 34556011 -118108992
174 13124547 33894127 -118177435
199 13129547 33724619 -118326045
224 13134547 34094771 -118030187
249 14281747 33646579 -117902609
274 17099147 34355400 -119060000
299 17053061 34450089 -119277534
324 17059050 34419017 -119296402
349 17071718 34291597 -119215150
374 17076371 34285792 -119286112
399 17082901 34276290 -119242263
424 17088082 34276556 -119221434
449 17093924 34280827 -119184952
474 17101256 34353550 -119070604
499 17108342 34275276 -119162139
524 17117469 34225441 -119193245
549 17124214 34243663 -119034397
574 17134043 34229754 -119041686
599 17138635 34226559 -119023719
624 17142856 34235797 -118985462
649 17148943 34194812 -119198166
674 17155428 34180959 -119218107
699 17161014 34217783 -119186066
724 17168364 34168089 -119179610
749 17174287 34149885 -119188224
774 17184035 3

In [112]:
# pick up the first 25 rows as a sample dataset
sample = data[0:25]
print(sample)

    parcelid  latitude  longitude
0   10754147  34144442 -118654084
1   10759547  34140430 -118625364
2   10843547  33989359 -118394633
3   10859147  34148863 -118437206
4   10879947  34194168 -118385816
5   10898347  34171873 -118380906
6   10933547  34131929 -118351474
7   10940747  34171345 -118314900
8   10954547  34218210 -118331311
9   10976347  34289776 -118432085
10  11073947  34265214 -118520217
11  11114347  34447747 -118565056
12  11116947  34465048 -118568166
13  11142747  34416889 -118505805
14  11193347  34585014 -118162010
15  11215747  34563376 -118019104
16  11229347  34526913 -118050581
17  11287347  34690736 -118135225
18  11288547  34733960 -118139298
19  11324547  34560018 -118169806
20  11391347  33986910 -118329553
21  11395747  33960238 -118319986
22  11404347  33959896 -118350438
23  11405747  33952952 -118361711
24  11417147  33931500 -118352104


In [115]:
size = len(sample)
print(size)

25


In [114]:
for row in sample.itertuples():
    if (row[0] == size - 1):
        print(row[0])

24


In [116]:
row 

Pandas(Index=24, parcelid=11417147, latitude=33931500, longitude=-118352104)

In [117]:
for attribute in row:
    print(attribute)

24
11417147
33931500
-118352104


### How to build a json format item

In [105]:
test = {
        'Name': '10759547',
        'Attributes': [
            {
                'Name': 'latitude',
                'Value': '34140430',
                'Replace': True
            },
            {
                'Name': 'longitude',
                'Value': '-118625364',
                'Replace': True
            },
        ]
    }

In [106]:
group = []

In [107]:
group.append(test)

In [108]:
group

[{'Attributes': [{'Name': 'latitude', 'Replace': True, 'Value': '34140430'},
   {'Name': 'longitude', 'Replace': True, 'Value': '-118625364'}],
  'Name': '10759547'}]

In [109]:
test2 = {
        'Name': '00000',
        'Attributes': [
            {
                'Name': 'latitude',
                'Value': '00000',
                'Replace': True
            },
            {
                'Name': 'longitude',
                'Value': '00000',
                'Replace': True
            },
        ]
    }

In [110]:
group.append(test2)

In [111]:
group

[{'Attributes': [{'Name': 'latitude', 'Replace': True, 'Value': '34140430'},
   {'Name': 'longitude', 'Replace': True, 'Value': '-118625364'}],
  'Name': '10759547'},
 {'Attributes': [{'Name': 'latitude', 'Replace': True, 'Value': '00000'},
   {'Name': 'longitude', 'Replace': True, 'Value': '00000'}],
  'Name': '00000'}]

### Test generating items using sample dataset

In [162]:
# generate one item
def generate_one_item(row):
    # index, parcelid, latitude, longitude
    parcelid = str(row[1])
    latitude = str(row[2])
    longitude = str(row[3])
    item = {
                'Name': parcelid,
                'Attributes': [
                    {
                        'Name': 'latitude',
                        'Value': latitude,
                        'Replace': True
                    },
                    {
                        'Name': 'longitude',
                        'Value': longitude,
                        'Replace': True
                    },
                ]
            }
    return item

In [163]:
generate_one_item(row)

{'Attributes': [{'Name': 'latitude', 'Replace': True, 'Value': '33931500'},
  {'Name': 'longitude', 'Replace': True, 'Value': '-118352104'}],
 'Name': '11417147'}

In [134]:
# put_items function test
def put_items(sdb, domainName, data):
    numOfItems = len(data)
    items_per_batch = []
    for row in data.itertuples():
        items_per_batch.append(generate_one_item(row))
        # put 25 items each time
        if (row[0] % 25 == 24) or (row[0] == numOfItems - 1):
            put_items_one_batch(sdb, domainName, items_per_batch)
            # reset items to []
            # items_per_batch = []
    return items_per_batch
            
# Do nothing here just for testing
def put_items_one_batch(sdb, domainName, items_per_batch):
#     response = sdb.batch_put_attributes(
#         DomainName=domainName,
#         Items=items_per_batch
#     )

    return

In [159]:
sdb = None
domainName = "ZillowData"
items_per_batch = put_items(sdb, domainName, sample)

In [143]:
print("Start: " + str(row[0] // 25 + 1) + " Put")

Start: 1 Put


In [161]:
type(items_per_batch)

list

# 3. sample queries connecting to DBaas

In [7]:
import json
import boto3
from pandas import DataFrame

In [2]:
# Config.json location
configFile = 'config.json'
with open(configFile) as globalSettings:
    config = json.load(globalSettings)

# read keys
ACCESS_KEY = str(config['AWSAccess'])
SECRET_KEY = str(config['AWSSecret'])
region = str(config['region'])

In [3]:
# Connect to SDB
Session = boto3.Session(
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name=region
)
sdb = Session.client('sdb')

In [181]:
response = sdb.select(
    SelectExpression="select latitude from ZillowData where latitude > '34171873'",
#     NextToken='string',
#     ConsistentRead=True|False
)
print(response)

{'ResponseMetadata': {'HTTPStatusCode': 200, 'RequestId': '0ad2b05f-b744-7163-36ce-3ff9ed52ac5f', 'BoxUsage': '0.0009278836', 'RetryAttempts': 0, 'HTTPHeaders': {'vary': 'Accept-Encoding', 'date': 'Wed, 05 Jul 2017 21:31:18 GMT', 'transfer-encoding': 'chunked', 'content-type': 'text/xml', 'server': 'Amazon SimpleDB', 'connection': 'keep-alive'}}, 'Items': [{'Name': '10879947', 'Attributes': [{'Name': 'latitude', 'Value': '34194168'}]}, {'Name': '10954547', 'Attributes': [{'Name': 'latitude', 'Value': '34218210'}]}, {'Name': '10976347', 'Attributes': [{'Name': 'latitude', 'Value': '34289776'}]}, {'Name': '11073947', 'Attributes': [{'Name': 'latitude', 'Value': '34265214'}]}, {'Name': '11114347', 'Attributes': [{'Name': 'latitude', 'Value': '34447747'}]}, {'Name': '11116947', 'Attributes': [{'Name': 'latitude', 'Value': '34465048'}]}, {'Name': '11142747', 'Attributes': [{'Name': 'latitude', 'Value': '34416889'}]}, {'Name': '11193347', 'Attributes': [{'Name': 'latitude', 'Value': '3458501

In [182]:
len(response['Items'])

100

In [5]:
response = sdb.select(
    SelectExpression="select * from ZillowData limit 5",
#     NextToken='string',
#     ConsistentRead=True|False
)
print(response)

{'ResponseMetadata': {'BoxUsage': '0.0000594282', 'HTTPHeaders': {'server': 'Amazon SimpleDB', 'transfer-encoding': 'chunked', 'date': 'Thu, 06 Jul 2017 22:13:25 GMT', 'connection': 'keep-alive', 'content-type': 'text/xml', 'vary': 'Accept-Encoding'}, 'HTTPStatusCode': 200, 'RetryAttempts': 0, 'RequestId': '7cf6c89b-daa0-b89e-ea78-03693e108332'}, 'NextToken': 'rO0ABXNyACdjb20uYW1hem9uLnNkcy5RdWVyeVByb2Nlc3Nvci5Nb3JlVG9rZW7racXLnINNqwMA\nC0kAFGluaXRpYWxDb25qdW5jdEluZGV4WgAOaXNQYWdlQm91bmRhcnlKAAxsYXN0RW50aXR5SURa\nAApscnFFbmFibGVkSQAPcXVlcnlDb21wbGV4aXR5SgATcXVlcnlTdHJpbmdDaGVja3N1bUkACnVu\naW9uSW5kZXhaAA11c2VRdWVyeUluZGV4TAANY29uc2lzdGVudExTTnQAEkxqYXZhL2xhbmcvU3Ry\naW5nO0wAEmxhc3RBdHRyaWJ1dGVWYWx1ZXEAfgABTAAJc29ydE9yZGVydAAvTGNvbS9hbWF6b24v\nc2RzL1F1ZXJ5UHJvY2Vzc29yL1F1ZXJ5JFNvcnRPcmRlcjt4cAAAAAAAAAAAAAAAAAUAAAAAAAAA\nAAAAAAAAAAAAAABwcHB4', 'Items': [{'Attributes': [{'Value': '34144442', 'Name': 'latitude'}, {'Value': '-118654084', 'Name': 'longitude'}], 'Name': '10754147'}, {'Attribu

In [30]:
items = response['Items']
id = []
latitudes = []
longitudes = []

for attr in items:
    lat = float(attr['Attributes'][0]['Value']) / 1000000
    long = float(attr['Attributes'][1]['Value']) / 1000000
    id.append(attr['Name'])
    latitudes.append(lat)
    longitudes.append(long)
    
data = DataFrame({'id': id, 'latitude': latitudes, 'longitude': longitudes})
data

,id,latitude,longitude
0,10754147,34.144442,-118.654084
1,10759547,34.140430,-118.625364
2,10843547,33.989359,-118.394633
3,10859147,34.148863,-118.437206
4,10879947,34.194168,-118.385816


In [15]:
data = data.set_index('id')
data

,latitude,longitude
id,,
10754147,34.144442,-118.654084
10759547,34.140430,-118.625364
10843547,33.989359,-118.394633
10859147,34.148863,-118.437206
10879947,34.194168,-118.385816


In [39]:
closest = data.sort_values(by=['latitude'], ascending=True)[0:10]
closest

,id,latitude,longitude
2,10843547,33.989359,-118.394633
1,10759547,34.140430,-118.625364
0,10754147,34.144442,-118.654084
3,10859147,34.148863,-118.437206
4,10879947,34.194168,-118.385816


In [45]:
# print(closest.to_json(orient='records'))
result = closest.to_dict(orient='records')
result

[{'id': '10843547', 'latitude': 33.989359, 'longitude': -118.394633},
 {'id': '10759547', 'latitude': 34.14043, 'longitude': -118.625364},
 {'id': '10754147', 'latitude': 34.144442, 'longitude': -118.654084},
 {'id': '10859147', 'latitude': 34.148863, 'longitude': -118.437206},
 {'id': '10879947', 'latitude': 34.194168, 'longitude': -118.385816}]

In [46]:
test = [<%= raw result %>]
test

SyntaxError: invalid syntax (<ipython-input-46-cdb3b90af9f2>, line 1)